In [10]:
import numpy as np
import pandas as pd

In [11]:
from google.colab import files
uploaded = files.upload()

In [12]:
data = pd.io.parsers.read_csv('ratings.dat', names=['user_id', 'movie_id', 'rating', 'time'],engine='python', delimiter='::')
movie_data = pd.io.parsers.read_csv('movies.dat',names=['movie_id', 'title', 'genre'],engine='python', delimiter='::')

In [13]:
ratings_mat = np.ndarray(shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),dtype=np.uint8)
ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

In [15]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

2) Compute SVD to get U, S and V. Use np.linalg.svd()

In [16]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

3. From your V.T select 50 components.
4. Implement a function that take movieID as input and then implement cosineimilarity along with 
sorting to recommend top 10 movies.

In [23]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

#print top n similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [24]:
k = 50
movie_id = 1 
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Georgia (1995)
Lamerica (1994)
Catwalk (1995)
400 Blows, The (Les Quatre cents coups) (1959)
Whatever It Takes (2000)
Chushingura (1962)
Way of the Gun, The (2000)
Grandfather, The (El Abuelo) (1998)
Teaching Mrs. Tingle (1999)


5. Repeat the same process except now instead of using SVD you will use PCA to get the eigenvectors.
6. You will require co-variance matrix as an input to your eig function.
Use np.cov() for getting co-variance matrix.
Use np.linalg.eig() for getting eigen vectors.

In [39]:
normalised_mat = ratings_mat - np.matrix(np.mean(ratings_mat, 1)).T
cov_mat = np.cov(normalised_mat)
evals, evecs = np.linalg.eig(cov_mat)

In [40]:
k = 50
movie_id = 1
top_n = 10

sliced = evecs[:, :k] 
top_indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, top_indexes)

Recommendations for Toy Story (1995): 

Toy Story (1995)
Georgia (1995)
Lamerica (1994)
Catwalk (1995)
400 Blows, The (Les Quatre cents coups) (1959)
Whatever It Takes (2000)
Chushingura (1962)
Way of the Gun, The (2000)
Grandfather, The (El Abuelo) (1998)
Teaching Mrs. Tingle (1999)


Couldn't see any major difference with results obtained from SVD and PCA